# Version 0.1

**Important information** 

This is a development version of the notebook reproducing the statistical analysis of the ORCHID trial. Even though the code producing the analyses and the plots will change and be refined, the libraries required to run the notebook will not change. Hence it can already be tested on computing platform to test that every thing works well regarding the packages installation and environment reproduction.

# ORCHID Clinical Trial

Multi-center, double blinded, randomized clinical trial conducted to assess the efficacy of hydroxychloroquine in the treatment of COVID-19. Results published in JAMA on November 9th 2020m, [paper available here](https://jamanetwork.com/journals/jama/fullarticle/10.1001/jama.2020.22240?utm_campaign=articlePDF&utm_medium=articlePDFlink&utm_source=articlePDF&utm_content=jama.2020.22240)

NHLBI made available the data to every authorized investigators. Hence, this notebook enables anybody with authorized credentials to reproduce the ORCHID clinical trial results by showing how to:
1. Access the data using the PIC-SURE API
2. Reproduce the results of this study using the open-source R programming languages

# Packages Installation 

In [ ]:
options(repr.plot.width=16, repr.plot.height=8)

In [ ]:
required_libraries <- c(
  'tidyverse',
  'devtools',
  'kableExtra',
  'survival',
  'survminer',
  'MASS',
  'quantreg',
  'DescTools', 
  'IRdisplay')
for (package in required_libraries) {
  if (!package %in% row.names(installed.packages())) install.packages(package)
  library(package, character.only = TRUE)
}
devtools::install_github("hms-dbmi/pic-sure-r-client", force = TRUE)
devtools::install_github("hms-dbmi/pic-sure-r-adapter-hpds", force = TRUE)

# Installing the library and connecting to the database 

User access authentication works through a security token, which is passed to the API using the token.txt file (file to be created by the user). In order to know how to get your PIC-SURE token, please see [the README of the PIC-SURE API GitHub repo](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/tree/master/NHLBI_BioData_Catalyst).

In [ ]:
# PICSURE_network_URL <- "https://biodatacatalyst.integration.hms.harvard.edu/picsure/"
PICSURE_network_URL <- "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
token_file <- "token.txt"
token <- scan(token_file, what = "character")
httr::set_config(httr::config(ssl_verifypeer=0L, ssl_verifyhost=0L))
myconnection <- picsure::connect(url = PICSURE_network_URL, token = token)
resource_id <- picsure::list.resources(myconnection)
resource <- hpds::get.resource(myconnection, resourceUUID = resource_id)

# Querying the data

In [ ]:
dictionary_results <- hpds::find.in.dictionary(resource, 'ORCHID', verbose = T)
list_variables <- hpds::extract.keys(dictionary_results)
query <- hpds::new.query(resource)
hpds::query.anyof.add(query, list_variables)
raw_df <- hpds::query.run(query, result.type = "dataframe") %>%
  as_tibble()

# Variable names 

In [ ]:
primary_outcome_name <- "d_covid15"
secondary_outcomes_binary <- c("d_mort15",
                               "d_mort29",
                               "d_ecmo_death")
secondary_outcomes_ccs <- c('d_covid3',
                            'd_covid8',
                            'd_covid29')
secondary_outcomes_daysfree <- c(
  'd_time_to_recovery',
  "d_hospfreedays",
  "d_oxyfreedays",
  "d_ventfreedays",
  "d_vasofreedays",
  "d_icufreedays"
)
safety_outcomes <- c(
  'safe_cytopenia',
  'safe_seizure',
  'safe_astalt',
  'safe_hypogly',
  'safe_ca',
  'safe_vtach'
)
covariates <- c('rand_trt',
                'bl_age',
                'bl_sex',
                'covid_ooscale_1',
                'd_sofa_gcs_s',
                'bl_symptomdt')

# Data Management

In [ ]:
simplified_names <- names(raw_df) %>%
  str_extract('((?<=X\\.COVID19\\.ORCHID\\.\\.\\.phs002299\\.\\.\\.)\\w+(?=\\.))|(Patient\\.ID)')
orchid_data <- raw_df[!is.na(simplified_names)]
names(orchid_data) <- simplified_names[!is.na(simplified_names)]

orchid_data$rand_trt <- relevel(orchid_data$rand_trt, 'Placebo')
orchid_data$safe_cytopenia <- apply(
  orchid_data[, c('safe_neutrop',
                  'safe_lympho',
                  'safe_anemia',
                  'safe_thombo')],
  1,
  function(row) ifelse(any(row == 'Yes', na.rm = TRUE), 'Yes', 'No'))


dm_factors <- function(factor) {
  factor <- as.character(factor)
  ifelse(factor == '', NA, ifelse(
    factor %in% c('Yes', '1'), 'Yes', ifelse(
      factor %in% c('No', '0'), 'No', NA
    )
  )) %>% as.factor()
}
orchid_data[c(safety_outcomes, secondary_outcomes_binary)] <- lapply(
  orchid_data[c(safety_outcomes, secondary_outcomes_binary)],
  dm_factors) %>%
  as_tibble()


create_ordered_daysfree <- function(variable) {
  factor_levels <- as.character(0:28)
  factor_variable <- factor(as.character(variable), ordered = TRUE, levels = factor_levels)
  return(factor_variable)
}
orchid_data[secondary_outcomes_daysfree] <- lapply(
  orchid_data[secondary_outcomes_daysfree], create_ordered_daysfree) %>%
  as_tibble()


create_ordered_covid_clinical_scale <- function(variable) {
  levels <- c(
    '1, Dead',
    '2, Hospitalized on invasive mechanical ventilation or ECMO',
    '3, Hospitalized on non-invasive ventilation or high flow nasal cannula',
    '4, Hospitalized on supplemental oxygen,',
    '5, Hospitalized not on supplemental oxygen',
    '6, Not hospitalized with limitation in activity (continued symptoms)',
    '7, Not hospitalized without limitation in activity (no symptoms)'
  )
  factor(variable, levels = levels, ordered = TRUE)
}
orchid_data[grep('^d_covid', names(orchid_data))] <- lapply(
  orchid_data[grep('^d_covid', names(orchid_data))],
  create_ordered_covid_clinical_scale) %>%
  as_tibble()

# Statistical Analysis

## COVID-19 Ordered Clinical Scale Description

In [ ]:
coos_df <- orchid_data %>%
  dplyr::select(subject_id, rand_trt, starts_with("d_covid")) %>%
  pivot_longer(cols = starts_with("d_covid"),
               names_to = "date",
               values_to = "COOS") %>%
  mutate(COOS =  fct_rev(COOS))

barplot_width <- 0.6
plot_count_COOS <- coos_df %>%
  filter(date %in% c("d_covid15", "d_covid29")) %>%
  mutate(rand_trt = fct_recode(rand_trt,
                               "Hydroxychloroquine\n(n=242)" = "Hydroxychloroquine",
                               "Placebo\n(n=237)" = "Placebo"),
         date = fct_recode(date,
                           "14 d After randomization\n(primary outcome)" = "d_covid15",
                           "28 d After randomization\n(secondary outcome)" = "d_covid29")) %>%
  ggplot(aes(x = rand_trt, fill = COOS)) +
  geom_bar(width = barplot_width, position="fill", colour = "black") +
  scale_y_continuous(labels = scales::percent,
                     limits = c(0, 1)) +
  facet_grid( ~ date) +
  scale_fill_brewer(palette = "Greens") +
  theme_bw() +
  theme(legend.background = element_blank(),
        legend.box.background = element_rect(colour = "black"),
        panel.spacing = grid::unit(c(0), "lines"),
        axis.line = element_line(colour = "black"),
        panel.grid.major = element_blank(),
        # panel.grid.minor = element_blank(),
        panel.border = element_blank(),
        panel.background = element_blank(),
        strip.background = element_rect(colour = "white", fill = "white")) +
  labs(title="Clinical Status on the Coronavirus Disease (COVID) Outcomes Scale 14 Days and 28 Days After Randomization",
       fill = "Clinical status (COVID Outcomes Scale category)"
       ) +
  xlab("Treatment group") +
  ylab("Patients with clinical status, %")
plot_count_COOS

In [ ]:
### Table count
table_count_coos <- coos_df %>%
  filter(date %in% c("d_covid15", "d_covid29")) %>%
  mutate(COOS = COOS) %>%
  group_by(COOS, date, rand_trt) %>%
  count() %>%
  pivot_wider(id_cols = COOS,
              names_from = c(date, rand_trt),
              values_from = n) %>%
  mutate(across(everything(), replace_na, replace = 0)) %>%
  rename("Hydroxychloroquine\n(n = 242)" = "d_covid15_Hydroxychloroquine",
         "Hydroxychloroquine\n(n = 242) " = "d_covid29_Hydroxychloroquine",
         "Placebo\n(n = 237)" = "d_covid15_Placebo",
         "Placebo\n(n = 237) " = "d_covid29_Placebo",
         "Clinical status\n(COVID Outcomes Scale category)" = "COOS")

table_count_coos %>%
  kableExtra::kbl() %>%
  kableExtra::kable_paper() %>%
  kableExtra::add_header_above(c(" ",
                     "14 d After randomization, No. (%)" = 2,
                     "28 d After randomization, No. (%)" = 2)
  ) %>%
as.character() %>%
display_html()

## Outcomes

In [ ]:

fitted_polr <- function(outcome, covariates, df) {
  formula <- paste(outcome,
                   paste(covariates, collapse = " + "),
                   sep = " ~ ")
  fitted_model <- MASS::polr(formula, Hess = TRUE, dat = df)
  return(fitted_model)
}

dm_polr_results <- function(fitted_model) {
  coef_table <- coef(summary(fitted_model))['rand_trtHydroxychloroquine',]
  OR <- exp(coef(fitted_model))['rand_trtHydroxychloroquine']
  coeff_names <- names(OR)
  ci <- exp(coef_table['Value'] + qnorm(c(0.025, 0.975))*coef_table['Std. Error'])
  p <- pnorm(abs(coef_table["t value"]), lower.tail = FALSE) * 2
  results <- round(c(OR, ci, p), 2)
  names(results) <- c('OR', '2.5%', '97.5%', 'pvalue')
  return(results)
}

ci_quantreg <- function(outcome, covariates, df) {
  formula <- paste0(outcome, ' ~ rand_trt')
  df[[outcome]] <- as.numeric(df[[outcome]])
  non_nan <- !is.na(df[[outcome]])
  qr_b <- boot.rq(cbind(1, df[['rand_trt']][non_nan]), df[[outcome]][non_nan], tau = 0.5, R = 10000)
  ci <- t(apply(qr_b$B, 2, quantile, c(0.025,0.975)))[2, ]
  return(ci)
}


list_results_outcomes <- list()
for (outcome_name in c(primary_outcome_name, secondary_outcomes_ccs, secondary_outcomes_daysfree)) {
  print(outcome_name)
  fitted_model <- fitted_polr(outcome_name, covariates, orchid_data)
  result_model_polyr <- dm_polr_results(fitted_model)
  adjusted_OR <- paste0(result_model_polyr[["OR"]], ' (', result_model_polyr[["2.5%"]], ' - ',
                        result_model_polyr[["97.5%"]], ')'
  )
  quartiles <- by(orchid_data, relevel(orchid_data$rand_trt, 'Hydroxychloroquine'),
     function(df) {
       numeric_var <- as.numeric(df[[outcome_name]])
       trunc(quantile(numeric_var, probs = c(0.25, 0.5, 0.75), na.rm = TRUE))
     })
  median <- quartiles %>%
    lapply(function(x) paste0(x[['50%']], ' (', x[['25%']], ' to ', x[['75%']], ')'))
  unadjusted_diff <- quartiles[[1]][['50%']] - quartiles[[2]][['50%']]
  if (unadjusted_diff == 0) {
    ci_unadjusted_diff <- as.character(unadjusted_diff)
  } else {
    ci_diff_num <- ci_quantreg(outcome = outcome_name, covariates = covariates, df = orchid_data)
    ci_unadjusted_diff <- paste0(unadjusted_diff, ' (', ci_diff_num[[1]], ' to ', ci_diff_num[[2]], ')')
  }
  list_results_outcomes[[outcome_name]] <- c(median,
                                                     "Unadjusted difference" = ci_unadjusted_diff,
                                                     "Adjusted odds ratio or odds ratio" = adjusted_OR
  )
}


dm_result_logistr <- function(fitted_model) {
  coef_table <- coef(summary(fitted_model))['rand_trtHydroxychloroquine',]
  OR <- exp(coef(fitted_model))['rand_trtHydroxychloroquine']
  ci <- exp(coef_table['Estimate'] + qnorm(c(0.025, 0.975))*coef_table['Std. Error'])
  results <- round(c(OR, ci), 2)
  names(results) <- c('OR', '2.5%', '97.5%')
  return(results)
}


# list_results_outcomes <- list()
for (safety_outcome in c(safety_outcomes, secondary_outcomes_binary)) {
  formula_safety <- paste(safety_outcome,
                          "rand_trt",
                          sep = " ~ ")
  non_na <- ! (is.na(orchid_data[[safety_outcome]]) | orchid_data[[safety_outcome]] == '')
  OR <- tryCatch({
    fitted_model <- glm(formula_safety, family = binomial(link = "logit"), data = orchid_data[non_na, ])
    OR <- dm_result_logistr(fitted_model)
    if (is.infinite(OR[['97.5%']])) OR <- NA
    OR
  },
  error = function(e) e,
  warning = function(w) NA
  )
  table_count <- table(orchid_data[non_na, c('rand_trt', safety_outcome)]) %>%
    as.data.frame() %>%
    pivot_wider(names_from = all_of(safety_outcome), values_from = 'Freq') %>%
    mutate(freq = round(Yes/(Yes + No), 3) * 100)
  table_count <- table_count[c(2, 1), ]
  count_safety = table_count[['Yes']]
  freq_safety = table_count[['freq']]
  names(count_safety) = table_count[['rand_trt']]
  names(freq_safety) = table_count[['rand_trt']]
  diff <- BinomDiffCI(x1 = table_count[[1, "Yes"]],
              n1 = sum(table_count[1, c('No', 'Yes')]),
              x2 = table_count[[2, "Yes"]],
              n2 = sum(table_count[2, c('No', 'Yes')]),
              method = "wald") %>% round(3) * 100
  list_results_outcomes[[safety_outcome]] <- c(
    paste0(count_safety, ' (', freq_safety, ')'),
    paste0(diff[[1]], ' (', diff[[2]], ' to ', diff[[3]], ')'),
    if (length(OR) == 1) NA else paste0(OR[['OR']], ' (', OR[['2.5%']], ' to ', OR[['97.5%']], ')')
  )
}


output_results <- do.call(rbind, list_results_outcomes)



list_renaming <- c(
  'safe_seizure' =	'Seizures',
  'safe_vtach' =	'Ventricular tachyarrhythmia',
  'safe_ca' =	'Cardiac arrest treated with CPR',
  'safe_astalt' =	'AST or ALT ≥2 times upper limit of normal',
  'safe_hypogly' =	'Symptomatic hypoglycemia',
  'd_covid3' =	'COVID Outcomes Scale score, median (IQR) at 2d',
  'd_covid8' =	'COVID Outcomes Scale score, median (IQR) at 2d',
  'd_covid15' =	'COVID Outcomes Scale score, median (IQR) at 2d',
  'd_covid29' =	'COVID Outcomes Scale score, median (IQR) at 2d',
  'd_mort15' =	'All-cause, all-location death',
  'd_mort29' =	'All-cause, all-location death',
  'd_ecmo_death' =	'Ecmo or Death',
  'd_hospfreedays' =	'Hospital Free Days',
  'd_icufreedays' =	'ICU Free Days',
  'd_oxyfreedays' =	'Oxygen free days',
  'd_time_to_recovery' =	'Time to recovery',
  'd_vasofreedays' =	'Vasopressor Free Days',
  'd_ventfreedays' =	'Ventilator Free Days'
)
row.names(output_results) <- list_renaming[row.names(output_results)]

output_results %>%
  kable(caption="Effect of Hydroxychloroquine on COVID Outcomes Scale category") %>%
  kable_minimal() %>%
  as.character() %>%
  IRdisplay::display_html()

## Survival Outcomes

In [ ]:

orchid_data$vs_died[orchid_data$vs_died == ''] <- NA
orchid_data$vs_died <- fct_recode(orchid_data$vs_died, '0' = 'No', '1' = 'Yes') %>%
  as.character() %>%
  as.numeric()

orchid_data$death_fu <- orchid_data$d_lastalivedt + 1

coxph_death <- coxph(Surv(time = orchid_data$death_fu,
                          event = orchid_data$vs_died) ~
                       rand_trt + bl_age + bl_sex + covid_ooscale_1 + d_sofa_gcs_s + bl_symptomdt,
                     data = orchid_data)


survfit_death <- survfit(Surv(time = orchid_data$death_fu,
                              event = orchid_data$vs_died) ~ rand_trt,
                         data = orchid_data)


# Time to discharge

orchid_data$iho_dischyn[orchid_data$iho_dischyn == ''] <- NA
orchid_data$iho_dischyn <- fct_recode(orchid_data$iho_dischyn, '0' = 'No', '1' = 'Yes') %>%
  as.character() %>%
  as.numeric()


orchid_data$event_censored_dt <- apply(data.frame(orchid_data$iho_dischdt,
                 orchid_data$vs_deathdt,
                 orchid_data$vs_alivedt), 1, min, na.rm = TRUE)
# derived_vars$day_free_hospital


orchid_data$death_discharge <- as.factor(ifelse(orchid_data$vs_died == 0 | is.na(orchid_data$vs_died),
                                                ifelse(orchid_data$iho_dischyn != 1 | is.na(orchid_data$iho_dischyn),
                                                       "censor", 'discharge'),
                                                "death"))

discharge_data <- finegray(Surv(orchid_data$event_censored_dt, orchid_data$death_discharge) ~ .,
                           data = orchid_data,
                           etype = "discharge")

discharge_survival <- coxph(Surv(fgstart, fgstop, fgstatus) ~
                              rand_trt + bl_age + bl_sex + covid_ooscale_1 + d_sofa_gcs_s + bl_symptomdt,
                            data = discharge_data,
                            weight = fgwt)
survfit_discharge <- survfit(
  Surv(orchid_data$iho_dischdt, orchid_data$iho_dischyn) ~ rand_trt,
  data = orchid_data)


death_plot <- ggsurvplot(survfit_death,
           xlim = c(0, 29),
           size = 1,                 # change line size
           palette = c("#E7B800", "#2E9FDF"),# custom color palettes
           conf.int = FALSE,          # Add confidence interval
           pval = TRUE,              # Add p-value
           risk.table = TRUE,        # Add risk table
           risk.table.col = "strata",# Risk table color by groups
           legend.labs =c("Hydroxychloroquine", "Placebo"),
           risk.table.height = 0.25,
           ggtheme = theme_bw(),
           combine = TRUE,
           keep_data = TRUE,
           type="cuminc",
           title = "Treatment Effect on Survival"
)

death_plot

In [ ]:
discharge_plot <- ggsurvplot(
  survfit_discharge,
  xlim = c(0, 29),
  size = 1,                 # change line size
  palette = c("#E7B800", "#2E9FDF"),# custom color palettes
  conf.int = FALSE,          # Add confidence interval
  pval = TRUE,              # Add p-value
  risk.table = TRUE,        # Add risk table
  risk.table.col = "strata",# Risk table color by groups
  legend.labs =c("Hydroxychloroquine", "Placebo"),
  risk.table.height = 0.25,
  ggtheme = theme_bw(),
  combine = TRUE,
  keep_data = TRUE,
  linetype = 'strata',
  fun = 'event',
  type="cuminc",
  title = "Treatment Effect on Survival"
)

discharge_plot
